##   Code for running 3DVAR with the Lorenz 63 model.
#### Code developed by Greg Hakim, Ryan Torn, Aneesh Subramanian.

In [ ]:
import numpy as np
import time
from numpy.linalg import inv
import netCDF4 as nc
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import lorenz63_model as lor

###  Data assimilation experiment parameters

In [ ]:
assim_len = 1.0  #  Time between observations
fcst_len  = 2.0  #  Forecast length
nassim    = 200  #  Number of assimilation times
alpha     = 4.e-3  # Alpha control (how fast you try to converge to analysis)

H = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])  #  observation operator
#H = np.array([[1., 0., 0. ]])  #  observation operator for single observation
nobs = len(H[:,0])
R = np.eye(nobs) * 1.0e-2  #  observation error
#R = np.array([[1.0e-2]])  #  observation error for single observation

time1 = time.time()

In [ ]:
np.random.seed(0)

bfile = nc.Dataset('L63_B.nc')
invB = inv(bfile.variables['B_matrix'][:,:])
invR = inv(R)

###  Create arrays

In [ ]:
xb = np.empty(3)
xf = np.empty(3)
xe = np.empty(3)

yobs  = np.empty(nobs)
innov = np.empty(nobs)

xaerr = np.empty((nassim, 3))
xberr = np.empty((nassim, 3))
xferr = np.empty((nassim, 3))

Jfin = np.empty(nassim)

### IC for truth taken from last time (column vector):

In [ ]:
xt = np.array(lor.advance(10., 20., 30., 100.))

### Populate initial state by perturbing true state

In [ ]:
xa = np.empty(3)
xa = xt[:] + np.random.normal(0, 0.1, 3)

In [ ]:
for t in range(nassim):

  #  Advance analysis to next assimilation time
  xb[0], xb[1], xb[2] = lor.advance(xa[0], xa[1], xa[2], assim_len)

  #  Advance the truth, compute observations at the next time
  xt[0], xt[1], xt[2] = lor.advance(xt[0], xt[1], xt[2], assim_len)
  yobs[:] = np.matmul(H,xt) + np.random.normal(0, np.diag(np.sqrt(R)), nobs)

  xa[:] = xb[:]
  niters = 0
  maxiter = 100
  Jold = 1.0e6
  J = 0.
  while abs(Jold - J) > 1.0e-5:

    Jold = J

    #  Compute innovation, background and observation cost function
    innov[:] = yobs[:] - np.matmul(H,xa)
    Jb = 2.0 * np.matmul(np.matmul(np.transpose(xa - xb), invB), xa - xb)
    J0 = 2.0 * np.matmul(np.matmul(np.transpose(innov), invR), innov)
    J = Jb + J0

    print('   cost function = ',J,", Error: ",np.sqrt(np.sum((xa[:]-xt[:])**2)))

    #  Compute the gradient in the cost function
    gJ = 2.0 * np.matmul(invB,xa - xb) - 2.0 * np.matmul(np.matmul(np.transpose(H),invR),innov)

    #  Compute the new state vector based on cost function gradient
    if niters == 0:
      xa[:] = xa[:] - alpha*gJ[:]
      cgJo = gJ[:]
    else:
      beta = np.matmul(np.transpose(gJ),gJ) / np.matmul(np.transpose(gJo),gJo)
      cgJ = gJ[:] + beta*cgJo[:]
      xa[:] = xa[:] - alpha*cgJ[:]
      cgJo = cgJ[:]

    gJo = gJ[:]

    niters = niters + 1

  print('final cost = ', J, ' after ', niters, ' iterations')

  Jfin[t] = J

  #  Compute analysis and background forecast error
  xberr[t,:] = xb[:] - xt[:]
  xaerr[t,:] = xa[:] - xt[:]

  # compute forecast and error
  xf[0], xf[1], xf[2] = lor.advance(xa[0], xa[1], xa[2], fcst_len)
  xe[0], xe[1], xe[2] = lor.advance(xt[0], xt[1], xt[2], fcst_len)
  xferr[t,:] = xf[:] - xe[:]

print('Analysis Error: ',np.sqrt(sum(sum(xaerr[:,:] * xaerr[:,:])) / float(nassim*3)))
print('Background Error: ',np.sqrt(sum(sum(xberr[:,:] * xberr[:,:])) / float(nassim*3)))
print('Forecast Error: ',np.sqrt(sum(sum(xferr[:,:] * xferr[:,:])) / float(nassim*3)))

time2 = time.time()

print("Total Time:",time2-time1)